CNN med Pytorch

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random
import os
import sys
from pathlib import Path
import h5py

#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split
import wandb

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

In [2]:
module_path = str(Path.cwd().parents[0].parents[0] / "methods")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [3]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

# print(bhArray.shape)
# print(sphArray.shape)

In [4]:
#Kombinerer dataene for å kunne kjøre gjennom modellen på et samlet datasett
dataArray = np.concatenate((bhArray,sphArray),axis=0)

# np.shape(dataArray)

In [5]:
# Labeler tabelle med 1 og 0 (0 = svart hull, 1 = spahleron)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

Sjekk om køyrer på GPU eller CPU

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [7]:
#Split data 75% i train og 25% i test
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [8]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [9]:
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [10]:
trainLoader = DataLoader(train, shuffle=True, batch_size=50)
testLoader = DataLoader(test, shuffle=True, batch_size=50)

Her er vårt CNN modell

In [11]:
# Her kommer modellen inn
class ConvModel(nn.Module):
    def __init__(self, dropout):

        super(ConvModel, self).__init__()
        # 2 cov lag blir opprette. Bildene har x, y og z verdi. 
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, padding=0)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=0)

        self.fc1 = nn.Linear(3*3*256, 128)
        self.fc2 = nn.Linear(128,2)

        self.dropout = nn.Dropout(dropout)



    def forward(self, x:Tensor):
        x = self.conv1(x)
        x = F.relu(x) #to activate function above

        x = F.max_pool2d(x,2)

        x = self.conv2(x)
        x = F.relu(x)

        x = F.max_pool2d(x,2)

        x = self.conv3(x)
        x = F.relu(x)

        x = F.max_pool2d(x,3)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
 
        return x

summere modellen sine parametera

In [12]:
from torchvision import models
from torchsummary import summary

model = ConvModel(0).to(device)
summary(model, (3, 50, 50))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 48, 48]             448
            Conv2d-2           [-1, 64, 22, 22]           9,280
            Conv2d-3            [-1, 256, 9, 9]         147,712
            Linear-4                  [-1, 128]         295,040
            Linear-5                    [-1, 2]             258
Total params: 452,738
Trainable params: 452,738
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.68
Params size (MB): 1.73
Estimated Total Size (MB): 2.43
----------------------------------------------------------------


In [13]:
def trainFunction(config=None):

    #init wandb
    with wandb.init(project="PyTorch", name="convmodel", config=config):
        #initialize variables
        model = ConvModel(config.dropout).to(device)

        # Teste dette ut: 
        criterion = nn.CrossEntropyLoss()
        def function_uno(x, y): 
            long_prob = -1.0 * F.log_softmax(x, 1)
            loss = long_prob.gather(1, y.unsqueeze(1))
            loss = loss.mean()
            return loss

        # Teste dette ut: (optimizer og learning rate)
        if config.optimizer is 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
        else:
            optimizer = torch.optim.SGD(model.parameters(), lr=config.learning_rate)
        #initialize variables end


        #data
        trainLoader = DataLoader(train, shuffle=True, batch_size=config.batch_size)
        testLoader = DataLoader(test, shuffle=True, batch_size=config.batch_size)
        #data end
        
        wandb.watch(model, function_uno, log='all')
        
        #START
        for epoch in range(8):

            #training variables
            trainRunningLoss = 0.0
            correct = 0
            y_pred = []
            y_true = []
            #training loop
            model.train()
            for i, data in enumerate(trainLoader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad() 
                output = model(inputs.permute(0,3,1,2)) # bytter på rekkefølge til dimensjonene 
                output1 = (torch.max(output.to(device), 1)[1]) # den predictede outputtn 
                y_pred.extend(output1) # Save Prediction
                
                y_true.extend(labels) # Save Truth
                loss = function_uno(output, labels.type(torch.LongTensor).to(device))
                loss.backward()
                optimizer.step() # Endrer på vekten 
                
                trainRunningLoss += loss.item() * inputs.size(0) #
            #training loop end
            correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()
            trainAccuracy = correct / len(y_true)
            #training variables end

            #test variables
            testRunningLoss = 0.0
            y_pred = []
            y_true = []
            #test loop
            model.eval()
            for j, data in enumerate(testLoader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                output = model(inputs.permute(0,3,1,2))# Feed Network

                output1 = (torch.max(output.to(device), 1)[1])
                y_pred.extend(output1) # Save Prediction
                
                y_true.extend(labels) # Save Truth
                loss = function_uno(output, labels.type(torch.LongTensor).to(device))
                testRunningLoss += loss.item() * inputs.size(0)
            #test loop end

            correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()
            testAccuracy = correct / len(y_true)
            testRunningLoss = testRunningLoss/len(bhArray)
            trainRunningLoss = trainRunningLoss/len(bhArray)
            #test variables end
            
            #Here comes print
            wandb.log({"Train epoch_loss":trainRunningLoss, "Test epoch_loss": testRunningLoss, "Train accuracy": trainAccuracy,"Test accuracy": testAccuracy})

        #END

<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\vlads\AppData\Local\Temp\ipykernel_14040\3335153958.py:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if config.optimizer is 'adam':


Denne biten kun for å sette opp prosjektet i wandb for første gang

In [14]:
# #Weights and biases set up
# wandb.init(
#     # set the wandb project where this run will be logged
#     name="convmodel",
#     project="PyTorch",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "epochs": 50,
#     'batch_size': 100,
#     }
# )

This piece starts script with hyperparameters given and then simulate training

In [15]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'Test accuracy'
        },
    'parameters': {
        'batch_size': {'values': [8]},
        'learning_rate': {'values': [0.01]},
        'optimizer': {'values': ['adam']},
        'dropout': {'values': [0.5]}
     }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project="PyTorch")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: k8r3yjlc
Sweep URL: https://wandb.ai/vladpus/PyTorch/sweeps/k8r3yjlc


In [16]:
wandb.agent(sweep_id, function=trainFunction, count=10)

wandb: Agent Starting Run: l2rcvfz9 with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0.5
wandb: 	learning_rate: 0.01
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vladcivilgin (vladpus). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Run l2rcvfz9 errored: AttributeError("'NoneType' object has no attribute 'dropout'")
wandb: ERROR Run l2rcvfz9 errored: AttributeError("'NoneType' object has no attribute 'dropout'")
wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
